# ChessVision Tests

In [1]:
import chessvision as cv
import cv_globals
from util import listdir_nohidden
import cv2
import numpy as np

test_data_dir = cv_globals.data_root + "test/"
filenames = listdir_nohidden(test_data_dir + "raw/")
test_imgs = np.array(list(map(lambda x: cv2.imread(test_data_dir + "raw/" + x), filenames)))

print(test_imgs.shape)
N = test_imgs.shape[0]

(8, 512, 512, 3)


In [2]:
from u_net import load_extractor
from square_classifier import load_classifier

classifier = load_classifier()

extractor = load_extractor()

Using Theano backend.


Loading square model..
Loading square model.. DONE
Loading board extraction model..
Loading board extraction model.. DONE


In [4]:
import os
#use this one to rough-label new test images
for i in range(N):
    filename = filenames[i]
    outfile = test_data_dir + "ground_truth/" + filename[:-4] + ".txt"
    
    if os.path.exists(outfile):
        continue

    _, _, chessboard, _, _ = cv.classify_raw(test_imgs[i], filename, extractor, classifier)
    res = vectorize_chessboard(chessboard)
    
    with open(outfile, "w") as out:
        out.write(str(res))

Processing image 6404e621-cc56-4db2-8970-76ea123e8a71.JPG
Extracting board..
Found 7 contour(s)
Filtered to 1 contour(s)
Extracting board.. DONE
Classifying board..
Predicted more than two white kings!
Classifying board.. DONE
Processing image 6404e621-cc56-4db2-8970-76ea123e8a71.JPG.. DONE


NameError: name 'vectorize_chessboard' is not defined

In [63]:
import ast
# use this one to compute test-accuracy
test_accuracy = 0
for i in range(N):
    filename = filenames[i]
    truth_file = test_data_dir + "ground_truth/" + filename[:-4] + ".txt"
    with open(truth_file) as truth:
        true_labels = ast.literal_eval(truth.read())

    _, _, chessboard, _, _ = cv.classify_raw(test_imgs[i], filename, extractor, classifier)
    res = vectorize_chessboard(chessboard)
    test_accuracy += sim(res, true_labels)
    
test_accuracy /= N
test_accuracy *= 100

Processing image f29c1937-8279-45a5-a74a-0c6b1aa3ffbe.JPG
Extracting board..
Found 5 contour(s)
Filtered to 1 contour(s)
Extracting board.. DONE
Classifying board..
Classifying board.. DONE
Processing image f29c1937-8279-45a5-a74a-0c6b1aa3ffbe.JPG.. DONE
Processing image 8991e2f3-56cc-4ae3-a57d-69f8c9586efe.JPG
Extracting board..
Found 3 contour(s)
Filtered to 1 contour(s)
Extracting board.. DONE
Classifying board..
Classifying board.. DONE
Processing image 8991e2f3-56cc-4ae3-a57d-69f8c9586efe.JPG.. DONE
Processing image 325732c8-7fb2-47a3-a579-e335c304d0af.JPG
Extracting board..
Extracting board.. DONE
Classifying board..
Classifying board.. DONE
Processing image 325732c8-7fb2-47a3-a579-e335c304d0af.JPG.. DONE
Processing image b7ab6da8-face-4c24-84ab-02ae722eac70.JPG
Extracting board..
Extracting board.. DONE
Classifying board..
Classifying board.. DONE
Processing image b7ab6da8-face-4c24-84ab-02ae722eac70.JPG.. DONE
Processing image 18bc9861-4199-4ea6-9e32-c15c8d05d349.JPG
Extracting

In [65]:
print("Test accuracy is {}%".format(test_accuracy))

Test accuracy is 99.0625%


In [36]:
res = vectorize_chessboard(chessboard)
print(res)

['f', 'f', 'f', 'f', 'f', 'R', 'f', 'K', 'P', 'P', 'f', 'f', 'f', 'f', 'P', 'P', 'f', 'f', 'P', 'R', 'f', 'Q', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'p', 'f', 'f', 'f', 'f', 'f', 'f', 'p', 'q', 'f', 'f', 'b', 'f', 'f', 'N', 'f', 'f', 'p', 'f', 'f', 'p', 'r', 'p', 'f', 'f', 'f', 'f', 'r', 'k', 'f', 'f']


In [5]:
def sim(a, b):
    return sum([aa == bb for aa, bb in zip(a, b)]) / len(a)

def vectorize_chessboard(board):
    res = ["f"] * 64
    
    piecemap = board.piece_map()
    
    for piece in piecemap:
        res[piece] = piecemap[piece].symbol()
        #print(piece)

    return res
        